In [1]:
import sqlite3
import geopandas as gpd
import pandas as pd

In [31]:
# Conectar ao GeoPackage usando sqlite3
conn = sqlite3.connect('/home/pedro/Documents/GIT_WORKSPACE/split_offdb/inputs/input.gpkg')
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite');")

,minx,miny,maxx,maxy
17,-37.0,-10.0,-36.5,-9.5


In [47]:
grid_gdf=gpd.read_parquet('/home/pedro/Documents/GIT_WORKSPACE/split_offdb/inputs/grid.parquet')
polygon = grid_gdf.loc[grid_gdf['grid_id'] == 18, 'geom'].iloc[0]
polygon_wkt=polygon.wkt
polygon_wkt


'POLYGON ((-37 -10, -37 -9.5, -36.5 -9.5, -36.5 -10, -37 -10))'

In [64]:
# Criar consulta SQL com ST_Intersects e WKT
sql_query = f"""
SELECT id, id_layer, AsText(geom) geom
FROM input
WHERE ST_Intersects(
    geom,
    ST_PolygonFromText('{polygon_wkt}', 4674)
);
"""
sql_query


"\nSELECT id, id_layer, AsText(geom) geom\nFROM input\nWHERE ST_Intersects(\n    geom,\n    ST_PolygonFromText('POLYGON ((-37 -10, -37 -9.5, -36.5 -9.5, -36.5 -10, -37 -10))', 4674)\n);\n"

In [68]:
# Carregar os resultados em um GeoDataFrame
gdf = gpd.read_postgis(sql_query, conn, geom_col="geom")

In [70]:
len(gdf)

130111